In [1]:
import pandas as pd
import numpy as np
import matplotlib as mt
import glob
import pyranges as pr
from pyranges import PyRanges
from io import StringIO
import numpy
import os
import optparse
os.chdir('/Users/robin.burns/Documents/012TransposableElements/002PCRArtifacts/003control/003lowcomplexity/highermapq')
files = glob.glob("*.teinsertions")
print(files)

['6069_pcrfree.teinsertions', '6069.teinsertions', '6191.teinsertions', '6966_pcrfree.teinsertions', '6184.teinsertions', '6969.teinsertions', '6046_pcrfree.teinsertions', '6969_pcrfree.teinsertions', '9412.teinsertions', '6191_pcrfree.teinsertions', '9452.teinsertions', '6124.teinsertions', '9559.teinsertions', '9888.teinsertions', '6046.teinsertions', '6024.teinsertions', '9888_pcrfree.teinsertions', '9412_pcrfree.teinsertions', '6104.teinsertions', '8242_pcrfree.teinsertions', '6124_pcrfree.teinsertions', '8242.teinsertions', '9917_pcrfree.teinsertions', '6184_pcrfree.teinsertions', '6104_pcrfree.teinsertions', '6966.teinsertions', '9917.teinsertions', '9452_pcrfree.teinsertions', '6024_pcrfree.teinsertions', '9559_pcrfree.teinsertions']


In [3]:
for f in files:
    mytes=pd.read_csv(f, header=None, sep='\t')
    mytes.head(n=5)
    mytes[1]=pd.Categorical(mytes[1],categories=['Chr1', 'Chr2', 'Chr3', 'Chr4', 'Chr5', 'ChrC','ChrM'])
    mytes=mytes.sort_values([1, 2])
    mytes=mytes.reset_index()
    f2=str(f)
    f2=f.replace('.teinsertions', '')
    mytes.columns = ['index','ugh','Chromosome','Start','blah','TE_seq','TEfamily','readtype', 'blah', 'coverage']
    mytes['End']=mytes['Start']+1
    mytes=mytes[['Chromosome','Start','End','TEfamily']]
    mytes.head(n=5)
    mytes=PyRanges(mytes)
    mycluster=mytes.cluster(strand=None, by='TEfamily', slack=10000, count=False, nb_cpu=1)
    mycluster.to_bed('%s.tmp'%(f2))
    mycluster=pd.read_csv('%s.tmp'%(f2), header=None, sep='\t')
    mygrp = mycluster.groupby(7) \
       .agg({0:'unique',1:'median', 6:'unique'}) \
       .reset_index()

    mygrp[0] = mygrp[0].str[0]
    mygrp[6] = mygrp[6].str[0]
    mygrp[1] = mygrp[1].astype(int)
    mygrp[2]=mygrp[1]+1
    mygrp=mygrp[[0,1,2,6]]
    mygrp.columns = ['Chromosome','Start','End',
                     'TEfamily']
    mygrp['acc']=f2
    mygrp.to_csv('%s.exons.bed'%(f2), header=True)
